# Notebook to calculate Inter Annotator Agreement

### Import the libraries

In [1]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.insert(0, '..')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations
from src.d03_inter_annotator_agreement.scoring_functions import create_scoring_matrix

from definitions import ROOT_DIR


## Small Tutorial

Most of the functions used in this tutorial are based on the "Corpus" class. Please follow this turorial before.

In [2]:
    
dataframe_dir = '/home/jkuettel/NLP_spark/data/02_processed_to_dataframe/preprocessed_dataframe_test.pkl'

stat_df = pd.read_pickle(dataframe_dir)
stat_df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:A layer:Instrumenttypes ...,[span id:B1 annotator:B layer:Policydesignchar...,,,,,
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[C, D]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:C layer:Instrumenttypes ...,[span id:D1 annotator:D layer:Policydesignchar...,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:B layer:Instrumenttypes...,[span id:C58 annotator:C layer:Instrumenttypes...,,,,
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:A layer:Policydesigncha...,[span id:B129 annotator:B layer:Policydesignch...,,,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C, D]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:B layer:Instrumenttype...,[span id:C165 annotator:C layer:Instrumenttype...,[span id:D27 annotator:D layer:Instrumenttypes...,,,


First create a object of class Inter_Annotator_Agreement. The constructor takes a stat_df as input, has a optional argument DEBUG where only the first 10 articles are taken to test different functions. Per default, "Front" and "Whereas" articles are excluded.



In [3]:
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False)
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)


In [4]:
test_evaluator.df.shape

(412, 13)

In [5]:
test_evaluator_debug.df.shape

(10, 13)

Inter_Annotator_Agreement is a child class of the Corpus class, so all methods of the Corpus class are available

In [6]:
test_dir = repository(policy = 'EU_32008R1099')
test_evaluator.get_span_list(conditional_rep = test_dir, annotators = 'annotators', item = 'tag', value =  'Tech_LowCarbon')

[span id:C3882 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:18 stop:25 text:nuclear,
 span id:C3883 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:95 stop:109 text:nuclear energy,
 span id:C3884 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:151 stop:158 text:nuclear,
 span id:C5161 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:125 stop:141 text:renewable energy,
 span id:C5162 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:393 stop:409 text:renewable energy,
 span id:C5163 annotator:C layer:Technologyandapplicationspecificity feature:TechnologySpecificity tag:Tech_LowCarbon start:499 stop:515 text:renewable energy,
 span id:C5164 annotator:C layer:Technologyandapplicationspecif

To calculate the inter annonator agreement, there are two options


## Append the score to dataframe

This method appends the inter-annotator agreement for each article based on a set of inter-annotator agreement measures. The scores can calculated in parallel, this is the recommended method for computationally intensive scores.

First, we only consider the articles where the curation is finished and at least two annotators are present:

In [7]:
test_evaluator.keep_only_finished_articles()

Define a list of scoring metrics

In [8]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_heuristic']

**append_total_score_per_article(scoring_metrics, parallel = False, ** optional_tuple_properties)**

This function calculates the individual score for each article and for each metric defined in scoring_metrics (Can be a list of metrics or a single metric). For each metric, a new column is appended to the dataframe, therefore the scores can be stored and don't have to be recalculated. To speed up the computation, the scores can be calculated in parallel using the pandarell library. The kwargs "optional_tuple_properties" are reserved for pygamma properties such as a dissimilarity matrix. 



In [9]:
test_evaluator.append_total_score_per_article(scoring_metrics)

100%|██████████| 412/412 [00:00<00:00, 874.04it/s]


In [10]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_heuristic_score': 0.5275951402554365,
 'f1_tokenwise_score': 0.5210605573521196}

In [11]:
test_evaluator.df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G,f1_exact_score,f1_tokenwise_score,f1_heuristic_score
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:A layer:Instrumenttypes ...,[span id:B1 annotator:B layer:Policydesignchar...,,,,,,0.218750,0.386441,0.281250
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[C, D]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:C layer:Instrumenttypes ...,[span id:D1 annotator:D layer:Policydesignchar...,,,,0.289157,0.401575,0.421687
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:B layer:Instrumenttypes...,[span id:C58 annotator:C layer:Instrumenttypes...,,,,,0.567308,0.578846,0.653846
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:A layer:Policydesigncha...,[span id:B129 annotator:B layer:Policydesignch...,,,,,,0.736842,0.875000,0.736842
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C, D]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:B layer:Instrumenttype...,[span id:C165 annotator:C layer:Instrumenttype...,[span id:D27 annotator:D layer:Instrumenttypes...,,,,0.420198,0.519207,0.544511


There is also a normal implementation which uses parallel

In [12]:
test_evaluator.append_total_score_per_article(scoring_metrics, parallel = True)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Appending Pygamma

Pygamma is a special case. To calculate the  Pygamma-IAA score, we have the option to feed in a dissimilarity matrix and a list of all the possible annotated spans. The dissimilarity matrix can be calculated via the create_scoring_matrix function. Each missmatch between annotated spans of the same tagset or layer are penalized less. For more details, check the explanation in the source code. This matrix can be fed into the function as kwargs. When specyfing no dissimilarity matrix, all the missmatches are penalized equaly

In [13]:
# create custom scoring matrix
# by setting soft tagset dissimilarity equal true, missmatches in the same tagset are penalized less
# if soft layer dissimilarity would be set true, missmatches in the same layer woudl be penalized less

category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_dissimilarity_penality = 0.5, soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)

In [14]:
test_evaluator.append_total_score_per_article(scoring_metrics = 'pygamma', category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix, soft = False, parallel=True)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will 

Checking out the dataframe now:

In [15]:
test_evaluator.df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G,f1_exact_score,f1_tokenwise_score,f1_heuristic_score,pygamma_score
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:A layer:Instrumenttypes ...,[span id:B1 annotator:B layer:Policydesignchar...,,,,,,0.218750,0.386441,0.281250,0.558086
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[C, D]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:C layer:Instrumenttypes ...,[span id:D1 annotator:D layer:Policydesignchar...,,,,0.289157,0.401575,0.421687,0.467312
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:B layer:Instrumenttypes...,[span id:C58 annotator:C layer:Instrumenttypes...,,,,,0.567308,0.578846,0.653846,0.681334
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:A layer:Policydesigncha...,[span id:B129 annotator:B layer:Policydesignch...,,,,,,0.736842,0.875000,0.736842,0.711477
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C, D]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:B layer:Instrumenttype...,[span id:C165 annotator:C layer:Instrumenttype...,[span id:D27 annotator:D layer:Instrumenttypes...,,,,0.420198,0.519207,0.544511,0.609871


### Get total score

In [16]:
test_evaluator.df['Tokens'][1][27].get_token_spans()

[span id:CUR75 annotator:Curation layer:Technologyandapplicationspecificity feature:ApplicationSpecificity tag:App_Other start:173 stop:199 text:fuel or energy consumption,
 span id:CUR76 annotator:Curation layer:Technologyandapplicationspecificity feature:EnergySpecificity tag:Energy_Other start:173 stop:177 text:fuel]

**test_evaluator.get_total_score_df(scoring_metrics = 'all', annotator = 'all', weight_by = 'Tokens')**

Calculates the total scores specified in scoring metrics of the dataframe, either for a specific annotator or for all the annotators. The scores can be weighted by {'no_weighting, 'Tokens', 'Spans'}. Note that this works only for scoring metrics that are already calculated. The default argument 'all' retrieves the score for all the scores that have bee apended to the dataframe in the previous step.


In [17]:
#Get the total score of the corpus calculated as a mean of all the individual article scores
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_heuristic_score': 0.5275951402554365,
 'f1_tokenwise_score': 0.5210605573521196,
 'pygamma_score': 0.5800521496262065}

In [18]:
#Get the total score of the corpus calculated as a mean of all the individual article scores weighted by the total number of tokens per article
test_evaluator.get_total_score_df(weight_by = 'Tokens')

{'f1_exact_score': 0.37492081552891304,
 'f1_heuristic_score': 0.4870123246432716,
 'f1_tokenwise_score': 0.47508012433593083,
 'pygamma_score': 0.5509027203379928}

In [19]:
#Get the total score of the corpus calculated as a mean of all the individual article scores weighted by the total number of spans per article
test_evaluator.get_total_score_df(weight_by = 'Spans')

{'f1_exact_score': 0.3874058515882508,
 'f1_heuristic_score': 0.5030915691250408,
 'f1_tokenwise_score': 0.4937102678853899,
 'pygamma_score': 0.5577922993339453}

if only specific scores are required:

In [20]:
test_evaluator.get_total_score_df(scoring_metrics = 'f1_exact', weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476}

or

In [21]:
test_evaluator.get_total_score_df(scoring_metrics =['f1_exact', 'f1_tokenwise'], weight_by = 'Spans')

{'f1_exact_score': 0.3874058515882508,
 'f1_tokenwise_score': 0.4937102678853899}

### Get total score per annotator

The same function can be used to retrieve the score of individual annotators. That is the weighted average of scores for all the articles the annotator has participated. 

In [22]:
test_evaluator.get_total_score_df(scoring_metrics = 'all', annotator = 'A', weight_by = 'no_weighting')

{'f1_exact_score': 0.3270815592714389,
 'f1_heuristic_score': 0.45133424018050017,
 'f1_tokenwise_score': 0.43023272654547534,
 'pygamma_score': 0.5323197025429526}

For all the annotators

In [24]:

for ann in test_evaluator.finished_annotators:
    print('annotator: ', ann)
    print(test_evaluator.get_total_score_df(annotator = ann, weight_by = 'Spans'))
    print('')

annotator:  B
{'f1_exact_score': 0.4427467808588547, 'f1_heuristic_score': 0.5771630720105967, 'f1_tokenwise_score': 0.566544431953864, 'pygamma_score': 0.5992518029388881}

annotator:  A
{'f1_exact_score': 0.32323742866301736, 'f1_heuristic_score': 0.4441804412622658, 'f1_tokenwise_score': 0.41010308682078017, 'pygamma_score': 0.515248078336602}

annotator:  D
{'f1_exact_score': 0.3071820274746298, 'f1_heuristic_score': 0.39638095203759655, 'f1_tokenwise_score': 0.37844268250236957, 'pygamma_score': 0.4851138537478046}

annotator:  C
{'f1_exact_score': 0.4381654829355101, 'f1_heuristic_score': 0.5468911489380965, 'f1_tokenwise_score': 0.5641211824549889, 'pygamma_score': 0.5972089434642157}



Or for specific scores

In [25]:
test_evaluator.get_total_score_df(annotator ='A', scoring_metrics = ['f1_exact', 'f1_tokenwise'], weight_by = 'Spans')


{'f1_exact_score': 0.32323742866301736,
 'f1_tokenwise_score': 0.41010308682078017}

### Rank articles by score

In [26]:
test_evaluator.df.sort_values(by=['f1_heuristic_score'])

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G,f1_exact_score,f1_tokenwise_score,f1_heuristic_score,pygamma_score
EU_32019L0944_Title_0_Chapter_2_Section_0_Article_04,,article 4\r\nfree choice of supplier\r\nmember...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, D]",[span id:CUR12640 annotator:Curation layer:Pol...,[span id:A4497 annotator:A layer:Policydesignc...,,[],[span id:D3030 annotator:D layer:Policydesignc...,,,,0.0,0.000000,0.0,0.004487
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_10,,article 10\r\nimplementing measures\r\n1. th...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR13100 annotator:Curation layer:Ins...,[span id:A4760 annotator:A layer:Instrumenttyp...,[span id:B7175 annotator:B layer:Instrumenttyp...,[span id:C6624 annotator:C layer:Policydesignc...,,,,,0.0,0.000000,0.0,0.194174
EU_32019L0944_Title_0_Chapter_4_Section_0_Article_39,,article 39\r\ncombined operator\r\narticle 35(...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, C]",[span id:CUR15331 annotator:Curation layer:Pol...,[span id:A5952 annotator:A layer:Policydesignc...,,[span id:C7222 annotator:C layer:Policydesignc...,,,,,0.0,0.000000,0.0,0.587669
EU_32006L0066_Title_0_Chapter_0_Section_0_Article_29,,article 29\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR10714 annotator:Curation layer:Pol...,,[span id:B6443 annotator:B layer:Policydesignc...,[span id:C5728 annotator:C layer:Policydesignc...,,[],,,0.0,0.000000,0.0,0.286936
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_08,,article 8\r\nannual nuclear statistics\r\nthe ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR6743 annotator:Curation layer:Inst...,[span id:A2293 annotator:A layer:Instrumenttyp...,[span id:B3594 annotator:B layer:Policydesignc...,[span id:C3873 annotator:C layer:Policydesignc...,,,,,0.0,0.000000,0.0,0.185382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EU_32009R0079_Title_0_Chapter_0_Section_0_Article_14,,article 14\r\namendments to directive 2007/46/...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[D, C]",[span id:CUR12469 annotator:Curation layer:Pol...,,,[span id:C6392 annotator:C layer:Policydesignc...,[span id:D2925 annotator:D layer:Policydesignc...,,,,1.0,1.000000,1.0,1.000000
EU_32019L0944_Title_0_Chapter_7_Section_5_Article_73,,article 73\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, D]",[span id:CUR12501 annotator:Curation layer:Pol...,[span id:A4492 annotator:A layer:Policydesignc...,,[],[span id:D2942 annotator:D layer:Policydesignc...,,,,0.0,0.806452,1.0,0.976612
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_54,,article 54\r\namendments to directive 2012/27/...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR9806 annotator:Curation layer:Poli...,[span id:A3223 annotator:A layer:Policydesignc...,[span id:B5863 annotator:B layer:Policydesignc...,,,,,,1.0,1.000000,1.0,1.000000
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nentry into force\r\nthis regulat...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR384 annotator:Curation layer:Polic...,[span id:A100 annotator:A layer:Policydesignch...,[span id:B243 annotator:B layer:Policydesignch...,[span id:C268 annotator:C layer:Policydesignch...,,,,,0.5,0.777778,1.0,0.977545


## Get total score based on a spanlist

The inter annotator agreement score can be also calculated from a spanlist. For all the spans present, it calculates the inter agreement scores for alle the articls with at least two valid annoations. Can be used to caluclate simmilarity to curation.

**get_score_spanlist(conditional_rep, annotators , scoring_metric, item = None, value = None, weight_by = 'Spans', ** optional_tuple_properties)**

The function is designed similar to the get_span_list function. A set of spans is selected by providing a conditional repository and a optional item and value. For all the spans present, it calculates the inter agreement scores for all the articles where two finsihed annotators are found. Returns the resulting spalist and the score. Can be used to caluclate simmilarity to curation or scores in different categories.



In [27]:
test_dir = repository.from_repository_name('EU_32008R1099_Title_0_Chapter_0_Section_0_Article_12')
span_list = test_evaluator.get_span_list(test_dir, ['A', 'B'])

In [28]:
span_list

[span id:A100 annotator:A layer:Policydesigncharacteristics feature:Time tag:Time_InEffect start:76 stop:110 text:20th day following its publication,
 span id:A101 annotator:A layer:Policydesigncharacteristics feature:Actor tag:Addressee_default start:239 stop:252 text:member states,
 span id:B243 annotator:B layer:Policydesigncharacteristics feature:Time tag:Time_InEffect start:76 stop:134 text:20th day following its publication in the official journal,
 span id:B244 annotator:B layer:Policydesigncharacteristics feature:Actor tag:Addressee_default start:239 stop:252 text:member states]

So this would be the equivalent spanlist. Retrieving the IAA score of such a spanlist

In [30]:
span_list, score = test_evaluator.get_score_spanlist(conditional_rep = test_dir, annotators = ['A', 'B'] , scoring_metric = 'f1_heuristic', weight_by = 'Spans')
print(span_list)
print(f"\nscore: {score}")

[span id:A100 annotator:A layer:Policydesigncharacteristics feature:Time tag:Time_InEffect start:76 stop:110 text:20th day following its publication, span id:A101 annotator:A layer:Policydesigncharacteristics feature:Actor tag:Addressee_default start:239 stop:252 text:member states, span id:B243 annotator:B layer:Policydesigncharacteristics feature:Time tag:Time_InEffect start:76 stop:134 text:20th day following its publication in the official journal, span id:B244 annotator:B layer:Policydesigncharacteristics feature:Actor tag:Addressee_default start:239 stop:252 text:member states]

score: 1.0


Or use this function to get scores in specific categories:

In [51]:
test_dir = repository.from_repository_name('EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11')

span_list_score, score = test_evaluator.get_score_spanlist(test_dir, annotators = ['B', 'C'], item = 'layer', value = 'Instrumenttypes', scoring_metric = 'f1_heuristic', weight_by = 'Spans')
print(*span_list_score, sep='\n')
print(f"\nscore: {score}")


span id:B28 annotator:B layer:Instrumenttypes feature:InstrumentType tag:VoluntaryAgrmt start:12 stop:26 text:joint projects
span id:B29 annotator:B layer:Instrumenttypes feature:InstrumentType tag:VoluntaryAgrmt start:164 stop:178 text:joint projects
span id:B30 annotator:B layer:Instrumenttypes feature:InstrumentType tag:Unspecified start:1662 stop:1676 text:support scheme
span id:B31 annotator:B layer:Instrumenttypes feature:InstrumentType tag:PublicInvt start:1707 stop:1721 text:investment aid
span id:B32 annotator:B layer:Instrumenttypes feature:InstrumentType tag:VoluntaryAgrmt start:1879 stop:1967 text:council of europe convention for the protection of human rights and fundamental freedoms
span id:B33 annotator:B layer:Instrumenttypes feature:InstrumentType tag:VoluntaryAgrmt start:1978 stop:2031 text:international conventions or treaties on human rights
span id:B34 annotator:B layer:Instrumenttypes feature:InstrumentType tag:VoluntaryAgrmt start:2835 stop:2848 text:joint projec

## Check closeness to curation

In the same spirit as calculating the IAA scores, we can check the closeness to the curation.

**append_score_to_curation(self, scoring_metrics, parallel = False, ** optional_tuple_properties)**

This method works very similar to get_total_score_df, but calculated the closeness to the curation for all the scoring_metrics defined in scoring_metrics. Again, the scores can be calculted in parallel. Appends al the scores for all annotators that contributed in tuples, where each element corresponds to a scoring metric. Again, the optional tuple properties are reserved for the pygamma score

In [6]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_heuristic']

In [7]:
test_evaluator.append_score_to_curation(scoring_metrics, parallel = False)

100%|██████████| 412/412 [00:01<00:00, 381.43it/s]


As before, we can specify a dissimilarity matrix for pygamma

In [ ]:
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)
test_evaluator.append_score_to_curation(scoring_metrics = 'pygamma', category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)

Checking out the dataframe:

In [9]:
test_evaluator.df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G,D_to_curation,A_to_curation,C_to_curation,B_to_curation
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:A layer:Instrumenttypes ...,[span id:B1 annotator:B layer:Policydesignchar...,,,,,,,"[0.547945205479452, 0.6575342465753425, 0.6072...",,"[0.2222222222222222, 0.25396825396825395, 0.23..."
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[C, D]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:C layer:Instrumenttypes ...,[span id:D1 annotator:D layer:Policydesignchar...,,,,"[0.2830188679245283, 0.3584905660377359, 0.434...",,"[0.6861313868613139, 0.7445255474452555, 0.898...",
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:B layer:Instrumenttypes...,[span id:C58 annotator:C layer:Instrumenttypes...,,,,,,,"[0.6113989637305699, 0.7253886010362695, 0.731...","[0.6737967914438502, 0.6737967914438502, 0.686..."
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:A layer:Policydesigncha...,[span id:B129 annotator:B layer:Policydesignch...,,,,,,,"[0.631578947368421, 0.7368421052631577, 0.8358...",,"[0.8888888888888888, 1.0, 0.955223880597015, 0..."
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C, D]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:B layer:Instrumenttype...,[span id:C165 annotator:C layer:Instrumenttype...,[span id:D27 annotator:D layer:Instrumenttypes...,,,,"[0.3448275862068966, 0.4137931034482759, 0.380...",,"[0.6057142857142856, 0.7771428571428571, 0.882...","[0.7553191489361701, 0.7765957446808509, 0.766..."


### Get individual score

To retrieve the scores, im simmilar fashion to get_total_score_df, we use

**get_to_curation_score(self, weight_by = 'Tokens')**

This retrieves the closeness to the curation for all the annotators that participated, weighted by one of the following methods: {'no_weighting, 'Tokens', 'Spans'}


In [10]:
test_evaluator.get_to_curation_score(weight_by = 'no_weighting')

{'A': {'f1_exact': 0.5759851342165275,
  'f1_heuristic': 0.6626746821425515,
  'f1_tokenwise': 0.6238340038467249,
  'pygamma': 0.6834120446522839},
 'B': {'f1_exact': 0.6832770638730249,
  'f1_heuristic': 0.752194329001551,
  'f1_tokenwise': 0.7437593155270148,
  'pygamma': 0.7560678350909289},
 'C': {'f1_exact': 0.6741487861941954,
  'f1_heuristic': 0.7422988818458282,
  'f1_tokenwise': 0.8242361997009233,
  'pygamma': 0.7352200844498143},
 'D': {'f1_exact': 0.43231750058721435,
  'f1_heuristic': 0.509642244445365,
  'f1_tokenwise': 0.4868209888810718,
  'pygamma': 0.5321682809107152}}

In [11]:
test_evaluator.get_to_curation_score(weight_by = 'Tokens')

{'A': {'f1_exact': 0.5478083927236341,
  'f1_heuristic': 0.6311571670202513,
  'f1_tokenwise': 0.587297552665045,
  'pygamma': 0.6607333219124917},
 'B': {'f1_exact': 0.6814903098765762,
  'f1_heuristic': 0.7407736247878691,
  'f1_tokenwise': 0.7326448377049134,
  'pygamma': 0.7509166636506706},
 'C': {'f1_exact': 0.6507138521164361,
  'f1_heuristic': 0.715843449602768,
  'f1_tokenwise': 0.7889866472475074,
  'pygamma': 0.725085215579862},
 'D': {'f1_exact': 0.3884944280696229,
  'f1_heuristic': 0.4462154109859259,
  'f1_tokenwise': 0.42693279125784217,
  'pygamma': 0.4845427933378982}}

In [12]:
test_evaluator.get_to_curation_score(weight_by = 'Spans')

{'A': {'f1_exact': 0.5467249839982933,
  'f1_heuristic': 0.6319074034563689,
  'f1_tokenwise': 0.5875171172675752,
  'pygamma': 0.6573717251246058},
 'B': {'f1_exact': 0.6837066473988437,
  'f1_heuristic': 0.7435422687861271,
  'f1_tokenwise': 0.7349808323207989,
  'pygamma': 0.752137875450059},
 'C': {'f1_exact': 0.6498717755317543,
  'f1_heuristic': 0.7154422487051841,
  'f1_tokenwise': 0.7921979597493631,
  'pygamma': 0.7229385699964296},
 'D': {'f1_exact': 0.3859744866634834,
  'f1_heuristic': 0.4416399481547173,
  'f1_tokenwise': 0.4267859242472768,
  'pygamma': 0.4797380549228169}}

### Get Total score

To retireve the total closeness to the curation score, we use

**get_to_curation_score_total(self, weight_by = 'Tokens')**

For each article, we take an average of the annotator-curation scores (For all the finsihed annotators). To get the total average, all the individual article averages are weighted by one of the following methods: {'no_weighting, 'Tokens', 'Spans'}


In [13]:
test_evaluator.get_to_curation_score_total(weight_by = 'no_weighting')

{'f1_exact': 0.5994190569879829,
 'f1_heuristic': 0.6742289960750816,
 'f1_tokenwise': 0.6772445872361056,
 'pygamma': 0.6840240702993081}

In [14]:
test_evaluator.get_to_curation_score_total(weight_by = 'Tokens')

{'f1_exact': 0.5725567750165721,
 'f1_heuristic': 0.6388021548917368,
 'f1_tokenwise': 0.6390382312127151,
 'pygamma': 0.6604844361216857}

In [15]:
test_evaluator.get_to_curation_score_total(weight_by = 'Spans')

{'f1_exact': 0.5746747169968692,
 'f1_heuristic': 0.6419866749325036,
 'f1_tokenwise': 0.6450279541354138,
 'pygamma': 0.660325341137374}

# Check scores in different categories

In [16]:
layers = ['Technologyandapplicationspecificity', 'Policydesigncharacteristics', 'Instrumenttypes' ]
repo = repository()

for l in layers:
    span_list, score = test_evaluator.get_score_spanlist(conditional_rep = repo, annotators = 'annotators' , item = 'layer', value = l,scoring_metric = 'f1_heuristic', weight_by = 'Spans')
    print(f"layer: {l}, len of spanlist: {len(span_list)}, score: {score}")
    

layer: Technologyandapplicationspecificity, len of spanlist: 9799, score: 0.49129379850132515
layer: Policydesigncharacteristics, len of spanlist: 18660, score: 0.5154005542648732
layer: Instrumenttypes, len of spanlist: 5548, score: 0.47970571741677825
